In [ ]:
%pip install numpy
%pip install matplotlib 
%pip install -U scikit-learn
%pip install -U tensorflow_datasets

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from time import time

In [ ]:
def plot_loss(history, optimiser, name):
    plt.figure(figsize=(8, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'{name}: Training and Validation Loss using Optimiser {optimiser}')
    plt.show()

def compare_loss(history1, history2, label1, label2, name):
    plt.figure(figsize=(8, 6))
    plt.plot(history1.history['loss'], label=f'Training Loss ({label1})')
    plt.plot(history1.history['val_loss'], label=f'Validation Loss ({label1})')
    plt.plot(history2.history['loss'], label=f'Training Loss ({label2})', linestyle='--')
    plt.plot(history2.history['val_loss'], label=f'Validation Loss ({label2})', linestyle='--')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'{name}: Training and Validation Loss Comparison')
    plt.show()

def compare_epoch_time(timings, label1, label2, name):
    first_epochs = [i for i in range(1, len(timings / 2)+1)]
    second_epochs = second_epochs
    first_timing = timings[0:first_epochs-1]
    second_timing = timings[10:-1]

    plt.figure(figsize = (8,10))
    plt.plot(first_epochs, first_timing, label=f'Epoch time: {label1}')
    plt.plot(second_epochs, second_timing, label=f'Epoch time: {label2}', linestyle='--')
    plt.xlabel('Epoch')
    plt.ylabel('Time (s)')
    plt.title(f'{name}: Epoch Execution Time Comparison')
    plt.show()

In [ ]:
# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = keras.utils.to_categorical(y_train, 10), keras.utils.to_categorical(y_test, 10)

In [ ]:

# Define the model architecture
deep_feed_forward_less_layers = keras.models.Sequential([
    keras.layers.Input(shape=(28, 28)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

deep_feed_forward_more_layers = keras.models.Sequential([
    keras.layers.Input(shape=(28, 28)),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

DEEP_FORWARD_LESS_NAME = "Deep Feed Forward (less layers/neutrones)"
DEEP_FORWARD_MORE_NAME = "Deep Feed Forward (more layers/neutrones)"

In [ ]:
convulational_less_layers = keras.models.Sequential([
    keras.layers.Input(shape=(28, 28, 1)),
    keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])


convulational_more_layers = keras.models.Sequential([
    keras.layers.Input(shape=(28, 28, 1)),
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

CONVULATIONAL_LESS_NAME = "Convulational (less layers/neutrones)"
CONVULATIONAL_MORE_NAME = "Convulational (more layers/neutrones)"

In [ ]:
# Adam optimiser
legacy_adam = tf.compat.v1.train.AdamOptimizer()

# Ada delta optimiser
legacy_adadelta = tf.compat.v1.train.AdadeltaOptimizer()

In [ ]:
class TimingCallback(tf.keras.callbacks.Callback):
    """Callback to record the timings and epoch

    Args:
        tf (Callback): Class callback.
    """
    timings = []

    def on_epoch_begin(self, epoch, logs=None):
        self.start_time = time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time() - self.start_time
        self.timings.append(epoch_time)

    def get_timings(self):
        timings = self.timings
        return timings

In [121]:
deep_feed_more_adam = deep_feed_forward_more_layers
deep_feed_more_adadelta = deep_feed_forward_more_layers

# Compile the model with the legacy optimizer

deep_feed_more_adam.compile(optimizer=legacy_adam, loss='categorical_crossentropy', metrics=['accuracy'])
deep_feed_more_adadelta.compile(optimizer=legacy_adadelta, loss='categorical_crossentropy', metrics=['accuracy'])

timing_callback = TimingCallback()

# Train the models
history_adam = deep_feed_more_adam.fit(x_train, y_train, batch_size=32, epochs=10, validation_split=0.2, callbacks=[timing_callback])

more_layers_epoch_time = timing_callback.get_timings()

history_adadelta = deep_feed_more_adadelta.fit(x_train, y_train, batch_size=32, epochs=10, validation_split=0.2)


 737/1500 [=============>................] - ETA: 6s - loss: 1.3651 - accuracy: 0.7506

In [ ]:
plot_loss(history=history_adam, optimiser="Adam", name=DEEP_FORWARD_MORE_NAME)
plot_loss(history=history_adadelta, optimiser="adadelta", name=DEEP_FORWARD_MORE_NAME)

compare_loss(history1=history_adam, history2=history_adadelta, label1="Adam", label2="Adadelta", name=DEEP_FORWARD_MORE_NAME)

In [ ]:
deep_feed_less_adam = deep_feed_forward_less_layers
deep_feed_less_adadelta = deep_feed_forward_less_layers

# Compile the model with the legacy optimizer

deep_feed_less_adam.compile(optimizer=legacy_adam, loss='categorical_crossentropy', metrics=['accuracy'])
deep_feed_less_adadelta.compile(optimizer=legacy_adadelta, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the models
history_adam = deep_feed_less_adam.fit(x_train, y_train, batch_size=32, epochs=10, validation_split=0.2, callbacks=[timing_callback])

less_layers_epoch_times = timing_callback.get_timings()

history_adadelta = deep_feed_less_adam.fit(x_train, y_train, batch_size=32, epochs=10, validation_split=0.2)


In [ ]:
plot_loss(history=history_adam, optimiser="Adam", name=DEEP_FORWARD_LESS_NAME)
plot_loss(history=history_adadelta, optimiser="adadelta", name=DEEP_FORWARD_LESS_NAME)

compare_loss(history1=history_adam, history2=history_adadelta, label1="Adam", label2="Adadelta", name=DEEP_FORWARD_LESS_NAME)

compare_epoch_time(less_layers_epoch_times, more_layers_epoch_time, "Less Layers and Neurones", "More Layers and Neurones", "Layer and Neurone")

print (less_layers_epoch_times)

print (more_layers_epoch_time)

In [ ]:
conv_more_layers_adam = convulational_more_layers
conv_more_layers_adadelta = convulational_more_layers

# Compile the model with the legacy optimizer
conv_more_layers_adam.compile(optimizer=legacy_adam, loss='categorical_crossentropy', metrics=['accuracy'])
conv_more_layers_adadelta.compile(optimizer=legacy_adadelta, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the models
history_adam = conv_more_layers_adam.fit(x_train, y_train, batch_size=32, epochs=10, validation_split=0.2)
history_adadelta = conv_more_layers_adadelta.fit(x_train, y_train, batch_size=32, epochs=10, validation_split=0.2)

plot_loss(history=history_adam, optimiser="Adam")
plot_loss(history=history_adadelta, optimiser="Adam")


